# Ноутбук на память о первых шагах в сверточных сетях.
смотреть с попкорном))

In [1]:
import torch
import torchvision
from torch import nn
import pandas as pd
import numpy as np
import time

In [2]:
# hyper params
batch_size = 64
num_epoch = 200

### Создаем DataLoader попутно предобрабатывая данные
- Предварительный смотр данных можно найти в VGG_like.ipynb

In [3]:
from torchvision import transforms, datasets

train_transform = transforms.Compose([
        transforms.Resize((260,260)),
        transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
test_transform = transforms.Compose([
        transforms.Resize((260,260)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

trainset = datasets.ImageFolder(root='../imagenette/imagenette2-320/train/', transform=train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)
testset = datasets.ImageFolder(root='../imagenette/imagenette2-320/val/', transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, #batch_size=batch_size,
                                         shuffle=False)

/opt/conda/lib/python3.6/site-packages/torchvision/transforms/transforms.py:704: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  "please use transforms.RandomResizedCrop instead.")


## Создаем конструктор ResNet-like сетей.

### В конструктор подается словарь с параметрами сети:

 - body_input - 
 - conv_layers - 

 - class_qty - кол-во классов.
 - print_dim - печатает параметры тензора на выходе из соответствующего слоя.

Примеры:
ResNet-18: 
block_qty = [2,2,2,2]
weight_reduction = False

ResNet-36: 
block_qty = [3,4,6,3]
weight_reduction = False

ResNet-50:
block_qty = [3,4,6,3]
weight_reduction = True

ResNet-101:
block_qty = [3,4,23,3]
weight_reduction = True

ResNet-152:
block_qty = [3,8,36,3]
weight_reduction = True

In [4]:
params = {
    'net_input': [224, 224, 3], # list height, width, channel
    'first_layer': [7, 64, 2, 3], # params of first conv_layer. [kernel_size , channel_qty, stride, padding]
    'first_maxpool': [3 ,2, 1], # params of first maxpool_layer. [kernel_size , stride, padding]
    'blocks_qty': [2,2,2,2], # list of lists. each layer should be list: qty conv layers, stride same or valid, padding
    'weight_reduction': False, # using block with less weights qty
    'class_qty': [10],
}

In [5]:
class ResNet_like(nn.Module):

    def __init__(self, params):
        self.net_input = params['net_input']
        self.first_layer = params['first_layer']
        self.first_maxpool = params['first_maxpool']
        self.blocks_qty = params['blocks_qty']
        self.class_qty = params['class_qty']
        self.weight_reduction = params['weight_reduction']
#         self.print_dim = params['print_dim']
        
#         self.conv1x1 = conv1x1(*args)
#         self.batchnorm = nn.BatchNorm2d(channels)
        
        super().__init__()
        
        def block_contruct(block, no_in_block, resolution=None):
            

            self.channel_out = 16*(2**block)
            print('Input channels [in, out]: ', self.channel_in, self.channel_out)
            self.module_dict['Block_%d_%d'%(block, no_in_block)].add_module(
                        name='Block_%d_Conv_%d_1'%(block, no_in_block), 
                        module=nn.Conv2d(
                        self.channel_in,
                        self.channel_out,
                        kernel_size=3, stride=1, padding=(1,1))
                        )
            self.channel_in = self.channel_out
            self.module_dict['Block_%d_%d'%(block, no_in_block)].add_module(
                        name='Block_%d_BN_%d_1'%(block, no_in_block),
                        module=nn.BatchNorm2d(self.channel_out)
                        )
            self.module_dict['Block_%d_%d'%(block, no_in_block)].add_module(
                        name='Block_%d_Relu_%d_1'%(block, no_in_block),
                        module=nn.ReLU()
                        )
            self.module_dict['Block_%d_%d'%(block, no_in_block)].add_module(
                        name='Block_%d_Conv_%d_2'%(block, no_in_block), 
                        module=nn.Conv2d(
                        self.channel_in,
                        self.channel_out,
                        kernel_size=3, stride=1, padding=1)
                        )
            self.module_dict['Block_%d_%d'%(block, no_in_block)].add_module(
                        name='Block_%d_BN_%d_2'%(block, no_in_block),
                        module=nn.BatchNorm2d(self.channel_out)
                        )
            print('Out channels [in, out]: ', self.channel_in, self.channel_out)
      
        def reduction_block_construct(block, no_in_block, resolution=None):
                
            self.channel_out = 16*(2**block)
            self.module_dict['Block_%d_%d'%(block, no_in_block)].add_module(
                        name='Block_%d_Conv_%d_1'%(block, no_in_block), 
                        module=nn.Conv2d(
                        self.channel_in,
                        self.channel_out,
                        kernel_size=3, stride=1, padding=1)
                        )
            
            
            self.channel_in = self.channel_out
            self.module_dict['Block_%d_%d'%(block, no_in_block)].add_module(
                        name='Block_%d_BN_%d_1'%(block, no_in_block),
                        module=nn.BatchNorm2d(self.channel_out)
                        )
            self.module_dict['Block_%d_%d'%(block, no_in_block)].add_module(
                        name='Block_%d_Relu_%d_1'%(block, no_in_block),
                        module=nn.ReLU()
                        )

            self.module_dict['Block_%d_%d'%(block, no_in_block)].add_module(
                        name='Block_%d_Conv_%d_2'%(block, no_in_block), 
                        module=nn.Conv2d(
                        self.channel_in,
                        self.channel_out,
                        kernel_size=3, stride=1, padding=1)
                        )
            self.module_dict['Block_%d_%d'%(block, no_in_block)].add_module(
                        name='Block_%d_BN_%d_2'%(block, no_in_block),
                        module=nn.BatchNorm2d(self.channel_out)
                        )
            self.module_dict['Block_%d_%d'%(block, no_in_block)].add_module(
                        name='Block_%d_Relu_%d_2'%(block, no_in_block),
                        module=nn.ReLU()
                        )
            self.channel_out = 16*(2**block)*4
            self.module_dict['Block_%d_%d'%(block, no_in_block)].add_module(
                        name='Block_%d_Conv_%d_3'%(block, no_in_block), 
                        module=nn.Conv2d(
                        self.channel_in,
                        self.channel_out,
                        kernel_size=3, stride=1, padding=1)
                        )
            self.module_dict['Block_%d_%d'%(block, no_in_block)].add_module(
                        name='Block_%d_BN_%d_3'%(block, no_in_block),
                        module=nn.BatchNorm2d(self.channel_out)
                        )
            self.channel_in = self.channel_out

        
        # initial main net
        self.first = nn.Sequential()
        self.channel_in = self.net_input[2]
        self.channel_out = self.first_layer[1]
        self.resolution = np.array([self.net_input[0], self.net_input[1]])
        
        # first conv block
        self.first.add_module(name='Block_1_Conv_1_1', module=nn.Conv2d(
                        self.channel_in, 
                        self.channel_out, 
                        kernel_size=self.first_layer[0], 
                        stride=self.first_layer[2], 
                        padding=self.first_layer[3]))
        self.channel_in = self.channel_out
        self.resolution = (self.resolution - self.first_layer[0] + 1 + (self.first_layer[3] * 2)) // self.first_layer[2]
        print(self.resolution)
        self.first.add_module(name='Block_1_MaxPool_1', module=nn.MaxPool2d(
                        kernel_size=self.first_maxpool[0],
                        stride=self.first_maxpool[1],
                        padding=self.first_maxpool[2]))
        self.resolution = (self.resolution - self.first_maxpool[0] + 1 + (self.first_maxpool[2] * 2)) // self.first_maxpool[1]
        print(self.resolution)
        
        self.module_dict = nn.ModuleDict()
        
        block=2
        for block_number in self.blocks_qty:
            for no_in_block in range(block_number):
                if self.weight_reduction:
                    self.module_dict['Block_%d_%d'%(block, no_in_block+1)] = nn.Sequential()
                    reduction_block_construct(block, no_in_block+1) #params for correct layers naming
                else:
                    self.module_dict['Block_%d_%d'%(block, no_in_block+1)] = nn.Sequential()
                    block_contruct(block, no_in_block+1)  #params for correct layers naming
                # add maxpool after each block
                if no_in_block + 1 == block_number and block - 1 != len(self.blocks_qty):
                    self.resolution = self.resolution // 2
            block+=1
#         self.resolution = self.resolution * 4
        self.out = nn.Sequential()
        if self.weight_reduction:
            self.linear_input = 32*(2**(len(self.blocks_qty)))*4
        else:
            self.linear_input = 32*(2**(len(self.blocks_qty)))
        self.out.add_module(name='output',module=nn.Linear(self.linear_input, self.class_qty[0]))
        
    def conv1x1(in_channels, out_channels):
        return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x = self.first(x)
        
        block = 2
        for block_number in self.blocks_qty:
            for no_in_block in range(block_number):
                x_out = self.module_dict['Block_%d_%d' % (block, no_in_block + 1)](x)
#                 print(x.shape, x_out.shape)
                if x.shape == x_out.shape:
                    x = torch.add(x_out, x)
                else:
#                     print(x.shape, x_out.shape)
#                     print(x.device)
#                     print(torch.mean(torch.abs(x)))
                    resize_weights = torch.ones([x_out.shape[1], x.shape[1], 1, 1], device=x.device)
                    resize_weights = resize_weights / (x.shape[1]) #  * x_out.shape[1]
                    x = nn.functional.conv2d(x, resize_weights, bias=torch.zeros(x_out.shape[1],device=x.device))
#                     print(torch.mean(torch.abs(x)))
                    x = torch.add(x_out, x)
                x = nn.functional.relu(x)
                # downsampling
                if no_in_block + 1 == block_number and block - 1 != len(self.blocks_qty):
                    x = nn.functional.max_pool2d(x, kernel_size=2, stride=2)
                elif no_in_block + 1 == block_number and block - 1 == len(self.blocks_qty):
#                     print(x.shape)
                    x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
#                     print(x.shape)
#             print(torch.mean(torch.abs(x)))
            block += 1
#         print(x)
        x = x.view(x.size(0), -1)
        #         print(x.shape)
        out = self.out(x)
        #         print(out.shape)
        return out


Инициализируем модель с через конструктор

In [6]:
model = ResNet_like(params)
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

[112 112]
[56 56]
Input channels [in, out]:  64 64
Out channels [in, out]:  64 64
Input channels [in, out]:  64 64
Out channels [in, out]:  64 64
Input channels [in, out]:  64 128
Out channels [in, out]:  128 128
Input channels [in, out]:  128 128
Out channels [in, out]:  128 128
Input channels [in, out]:  128 256
Out channels [in, out]:  256 256
Input channels [in, out]:  256 256
Out channels [in, out]:  256 256
Input channels [in, out]:  256 512
Out channels [in, out]:  512 512
Input channels [in, out]:  512 512
Out channels [in, out]:  512 512


In [7]:
# from torchvision.models import resnet18
# model = resnet18(num_classes = 10)
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

Загружаем модель на видеокарту.

In [8]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# device = 'cpu'

print(device)

model.to(device)

cuda:1


ResNet_like(
  (first): Sequential(
    (Block_1_Conv_1_1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (Block_1_MaxPool_1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (module_dict): ModuleDict(
    (Block_2_1): Sequential(
      (Block_2_Conv_1_1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (Block_2_BN_1_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (Block_2_Relu_1_1): ReLU()
      (Block_2_Conv_1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (Block_2_BN_1_2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Block_2_2): Sequential(
      (Block_2_Conv_2_1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (Block_2_BN_2_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (Block_2_Relu_2_1): ReLU()
      (Block_2_Conv_2_2)

In [9]:
def printgradnorm(self, grad_input, grad_output):
# #     print('Inside ' + self.__class__.__name__ + ' backward')
# #     print('Inside class:' + self.__class__.__name__)
# #     print('')
# #     print('grad_input: ', type(grad_input))
# #     print('grad_input[0]: ', type(grad_input[0]))
# #     print('grad_output: ', type(grad_output))
# #     print('grad_output[0]: ', type(grad_output[0]))
# #     print('')
# #     print('grad_input size:', grad_input[0].size())
# #     print('grad_output size:', grad_output[0].size())
    print('grad_input norm:', grad_input[0].norm()) 

## Основной тренировочный цикл и подсчет метрик.
    Основная метрика accuracy (топ1). Очень не хотелось бы получать ошибку на топ5 accuracy при 10 классах.

In [ ]:
for epoch in range(num_epoch):  # loop over the dataset multiple times
    
#     model.train()
    start_time = time.time()
    for i, data in enumerate(trainloader, 0):
        
        # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data
#         print(data[1])
        inputs, labels = data[0].to(device), data[1].to(device)
#         print(labels)

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # hook
#         model.module_dict.Block_3_1.Block_3_Conv_1_2.register_backward_hook(printgradnorm)
#         model.module_dict.Block_2_1.Block_2_BN_1_2.register_backward_hook(printgradnorm)
        
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
#         print('Loss: ', loss)
        loss.backward()
        optimizer.step()
    scheduler.step()
    
    #Accuracy train and val
    correct_train, correct_val = 0, 0
    total_train, total_val = 0, 0
    with torch.no_grad():
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            outputs = outputs.to(device)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            outputs = outputs.to(device)
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    end_time = time.time()
    print("Epoch {}/{}, Time: {:.2f} sec, Loss: {:.3f}, Accuracy_train: {:.3f}, Accuracy_val: {:.3f}".
          format(epoch+1, num_epoch, end_time - start_time, loss, correct_train/total_train, correct_val/total_val))
#     print('Epoch: ', epoch, ' loss:', loss[0], ' accuracy_val: ', accuracy(outputs, labels))

       
        # print statistics
#         running_loss += loss.item()
#         if i % 10 == 9:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, loss)) #running_loss / 2000
#             print('labels:',  labels)
#             print('outputs: ', outputs)
#             running_loss = 0.0

print('Finished Training')

Epoch 1/200, Time: 115.62 sec, Loss: 1.697, Accuracy_train: 0.324, Accuracy_val: 0.101
Epoch 2/200, Time: 115.80 sec, Loss: 2.687, Accuracy_train: 0.418, Accuracy_val: 0.098
Epoch 3/200, Time: 115.20 sec, Loss: 1.459, Accuracy_train: 0.469, Accuracy_val: 0.107
Epoch 4/200, Time: 115.14 sec, Loss: 1.813, Accuracy_train: 0.514, Accuracy_val: 0.112
Epoch 5/200, Time: 114.43 sec, Loss: 1.417, Accuracy_train: 0.541, Accuracy_val: 0.094
Epoch 6/200, Time: 114.59 sec, Loss: 2.127, Accuracy_train: 0.569, Accuracy_val: 0.098
Epoch 7/200, Time: 115.22 sec, Loss: 1.163, Accuracy_train: 0.598, Accuracy_val: 0.098
Epoch 8/200, Time: 114.31 sec, Loss: 1.769, Accuracy_train: 0.622, Accuracy_val: 0.098
Epoch 9/200, Time: 114.53 sec, Loss: 1.813, Accuracy_train: 0.639, Accuracy_val: 0.099
Epoch 10/200, Time: 114.86 sec, Loss: 1.774, Accuracy_train: 0.643, Accuracy_val: 0.121
Epoch 11/200, Time: 115.70 sec, Loss: 0.570, Accuracy_train: 0.669, Accuracy_val: 0.098
Epoch 12/200, Time: 115.84 sec, Loss: 1.0